# Knowledge Bases for Amazon Bedrock - End to end example using multiple s3 bucket(s) as data source(s)

This notebook provides sample code for building an empty OpenSearch Serverless (OSS) index, Amazon Bedrock knowledge base and ingest documents into the index from multiple data sources. Now you can add multiple data sources with a Knowledge base.


#### Notebook Walkthrough

A data pipeline that ingests documents (typically stored in multiple Amazon S3) into a knowledge base i.e. a vector database such as Amazon OpenSearch Service Serverless (AOSS) so that it is available for lookup when a question is received.

- Load the documents into the knowledge base by connecting your multiple s3 bucket (multiple data source). 
- Ingestion - Knowledge base will split them into smaller chunks (based on the strategy selected), generate embeddings and store it in the associated vectore store.

![data_ingestion.png](./images/data_ingestion.png)


#### Steps: 
- Create Amazon Bedrock Knowledge Base execution role with necessary policies for accessing data from multiple S3 and writing embeddings into OSS.
- Create an empty OpenSearch serverless index.
- Download documents
- Create Amazon Bedrock knowledge base
- Create data source(s) within knowledge base which will connect to corresponding Amazon S3
- For each data source, start ingestion jobs using KB APIs which will read data from s3, chunk it, convert chunks into embeddings using Amazon Titan Embeddings model and then store these embeddings in AOSS. All of this without having to build, deploy and manage the data pipeline.

Once the data is available in the Bedrock Knowledge Base then a question answering application can be built using the Knowledge Base APIs provided by Amazon Bedrock.


#### Pre-requisites
This notebook requires permissions to:
- create and delete Amazon IAM roles
- create, update and delete Amazon S3 buckets
- access Amazon Bedrock
- access to Amazon OpenSearch Serverless

If running on SageMaker Studio, you should add the following managed policies to your role:
- IAMFullAccess
- AWSLambda_FullAccess
- AmazonS3FullAccess
- AmazonBedrockFullAccess
- Custom policy for Amazon OpenSearch Serverless such as:
```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "aoss:*",
            "Resource": "*"
        }
    ]
}
```
<div class="alert alert-block alert-info">
<b>Note:</b> Please make sure to enable `Anthropic Claude 3` and `Anthropic Claude Instant` model access in Amazon Bedrock Console, as the notebook will use Anthropic Claude 3 Sonnet and Claude instant models for testing the knowledge base once its created.
</div>


## Setup
Before running the rest of this notebook, you'll need to run the cells below to (ensure necessary libraries are installed and) connect to Bedrock.

In [ ]:
%pip install -U opensearch-py==2.3.1
%pip install -U boto3==1.34.85
%pip install -U retrying==1.3.4

In [1]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import json
import os
import boto3
from botocore.exceptions import ClientError
import pprint
from utility import create_bedrock_execution_role, create_bedrock_execution_role_multi_ds, create_oss_policy_attach_bedrock_execution_role, create_policies_in_oss, interactive_sleep
import random
from retrying import retry
suffix = random.randrange(200, 900)

sts_client = boto3.client('sts')
boto3_session = boto3.session.Session()
region_name = boto3_session.region_name
bedrock_agent_client = boto3_session.client('bedrock-agent', region_name=region_name)
service = 'aoss'
s3_client = boto3.client('s3')
account_id = sts_client.get_caller_identity()["Account"]
s3_suffix = f"{region_name}-{account_id}"

# Now you can add multiple data sources to a Knowledge Base. For this notebook, we'll test Knowledge Base creation with TWO different data sources
bucket_name_1 = f'bedrock-kb-{s3_suffix}-1' # replace it with your first bucket name.
bucket_name_2 = f'bedrock-kb-{s3_suffix}-2' # replace it with your second bucket name.

# NOTE: You can have only one bucket name in below list if you don't want to use multiple data sources for Knowledge Base creation.
bucket_names=[bucket_name_1,bucket_name_2]
pp = pprint.PrettyPrinter(indent=2)

In [4]:
# Check if bucket exists, and if not create S3 bucket for knowledge base data source
for bucket_name in bucket_names:
    try:
        s3_client.head_bucket(Bucket=bucket_name)
        print(f'Bucket {bucket_name} Exists')
    except ClientError as e:
        print(f'Creating bucket {bucket_name}')
        s3bucket = s3_client.create_bucket(
            Bucket=bucket_name,
            # CreateBucketConfiguration={ 'LocationConstraint': region_name }
        )

Bucket bedrock-kb-us-east-1-017444429555-1 Exists
Bucket bedrock-kb-us-east-1-017444429555-2 Exists


## Create a vector store - OpenSearch Serverless index

### Step 1 - Create OSS policies and collection
Firt of all we have to create a vector store. In this section we will use *Amazon OpenSerach serverless.*

Amazon OpenSearch Serverless is a serverless option in Amazon OpenSearch Service. As a developer, you can use OpenSearch Serverless to run petabyte-scale workloads without configuring, managing, and scaling OpenSearch clusters. You get the same interactive millisecond response times as OpenSearch Service with the simplicity of a serverless environment. Pay only for what you use by automatically scaling resources to provide the right amount of capacity for your application—without impacting data ingestion.

In [5]:
import boto3
import time
vector_store_name = f'bedrock-sample-rag-{suffix}'
index_name = f"bedrock-sample-rag-index-{suffix}"
aoss_client = boto3_session.client('opensearchserverless')
bedrock_kb_execution_role = create_bedrock_execution_role_multi_ds(bucket_names=bucket_names)

bedrock_kb_execution_role_arn = bedrock_kb_execution_role['Role']['Arn']

In [6]:
# create security, network and data access policies within OSS
encryption_policy, network_policy, access_policy = create_policies_in_oss(vector_store_name=vector_store_name,
                       aoss_client=aoss_client,
                       bedrock_kb_execution_role_arn=bedrock_kb_execution_role_arn)
collection = aoss_client.create_collection(name=vector_store_name,type='VECTORSEARCH')

In [7]:
pp.pprint(collection)

{ 'ResponseMetadata': { 'HTTPHeaders': { 'connection': 'keep-alive',
                                         'content-length': '314',
                                         'content-type': 'application/x-amz-json-1.0',
                                         'date': 'Mon, 10 Jun 2024 16:59:36 '
                                                 'GMT',
                                         'x-amzn-requestid': 'da95e5f4-9088-48cf-92dc-ee17551f9d99'},
                        'HTTPStatusCode': 200,
                        'RequestId': 'da95e5f4-9088-48cf-92dc-ee17551f9d99',
                        'RetryAttempts': 0},
  'createCollectionDetail': { 'arn': 'arn:aws:aoss:us-east-1:017444429555:collection/dlukh8g4zwzhq8e5mmq3',
                              'createdDate': 1718038776199,
                              'id': 'dlukh8g4zwzhq8e5mmq3',
                              'kmsKeyArn': 'auto',
                              'lastModifiedDate': 1718038776199,
                             

In [8]:
# Get the OpenSearch serverless collection URL
collection_id = collection['createCollectionDetail']['id']
host = collection_id + '.' + region_name + '.aoss.amazonaws.com'
print(host)

dlukh8g4zwzhq8e5mmq3.us-east-1.aoss.amazonaws.com


In [9]:
import time
# wait for collection creation
# This can take couple of minutes to finish
response = aoss_client.batch_get_collection(names=[vector_store_name])
# Periodically check collection status
while (response['collectionDetails'][0]['status']) == 'CREATING':
    print('Creating collection...')
    interactive_sleep(30)
    response = aoss_client.batch_get_collection(names=[vector_store_name])
print('\nCollection successfully created:')
pp.pprint(response["collectionDetails"])

Creating collection...
Done!.........................

Collection successfully created:
[ { 'arn': 'arn:aws:aoss:us-east-1:017444429555:collection/dlukh8g4zwzhq8e5mmq3',
    'collectionEndpoint': 'https://dlukh8g4zwzhq8e5mmq3.us-east-1.aoss.amazonaws.com',
    'createdDate': 1718038776199,
    'dashboardEndpoint': 'https://dlukh8g4zwzhq8e5mmq3.us-east-1.aoss.amazonaws.com/_dashboards',
    'id': 'dlukh8g4zwzhq8e5mmq3',
    'kmsKeyArn': 'auto',
    'lastModifiedDate': 1718038800956,
    'name': 'bedrock-sample-rag-867',
    'standbyReplicas': 'ENABLED',
    'status': 'ACTIVE',
    'type': 'VECTORSEARCH'}]


In [10]:
# create opensearch serverless access policy and attach it to Bedrock execution role
try:
    create_oss_policy_attach_bedrock_execution_role(collection_id=collection_id,
                                                    bedrock_kb_execution_role=bedrock_kb_execution_role)
    # It can take up to a minute for data access rules to be enforced
    interactive_sleep(60)
except Exception as e:
    print("Policy already exists")
    pp.pprint(e)

Opensearch serverless arn:  arn:aws:iam::017444429555:policy/AmazonBedrockOSSPolicyForKnowledgeBase_564
Done!.......................................................


## Step 2 - Create vector index

In [11]:
# Create the vector index in Opensearch serverless, with the knn_vector field index mapping, specifying the dimension size, name and engine.
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth, RequestError
credentials = boto3.Session().get_credentials()
awsauth = auth = AWSV4SignerAuth(credentials, region_name, service)

index_name = f"bedrock-sample-index-{suffix}"
body_json = {
   "settings": {
      "index.knn": "true",
       "number_of_shards": 1,
       "knn.algo_param.ef_search": 512,
       "number_of_replicas": 0,
   },
   "mappings": {
      "properties": {
         "vector": {
            "type": "knn_vector",
            "dimension": 1536,
             "method": {
                 "name": "hnsw",
                 "engine": "faiss",
                 "space_type": "l2"
             },
         },
         "text": {
            "type": "text"
         },
         "text-metadata": {
            "type": "text"         }
      }
   }
}

# Build the OpenSearch client
oss_client = OpenSearch(
    hosts=[{'host': host, 'port': 443}],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=300
)


In [12]:
# Create index
try:
    response = oss_client.indices.create(index=index_name, body=json.dumps(body_json))
    print('\nCreating index:')
    pp.pprint(response)

    # index creation can take up to a minute
    interactive_sleep(60)
except RequestError as e:
    # you can delete the index if its already exists
    # oss_client.indices.delete(index=index_name)
    print(f'Error while trying to create the index, with error {e.error}\nyou may unmark the delete above to delete, and recreate the index')
    


Creating index:
{ 'acknowledged': True,
  'index': 'bedrock-sample-index-867',
  'shards_acknowledged': True}
Done!.......................................................


## Download data to ingest into our knowledge base.
We'll use the following data:
 - sythetic data stored in a local directory as first data source
 - download data from URLs

In [13]:
## First datasource - copied from local directory

!mkdir -p ./data_1

import shutil

filenames = ["./sythetic_data/octank_financial_10K.pdf"] #source files
data_root_1 = "./data_1/" # destination directory

for item in filenames:
    filename = os.path.basename(item)
    shutil.copyfile(item, os.path.join(data_root_1, filename))


# You can comment below lines if you don't want to use multiple data sources
## Second datasource - copied from URLs
!mkdir -p ./data_2

from urllib.request import urlretrieve
# source urls
urls = [
    'https://s2.q4cdn.com/299287126/files/doc_financials/2023/ar/2022-Shareholder-Letter.pdf',
    'https://s2.q4cdn.com/299287126/files/doc_financials/2022/ar/2021-Shareholder-Letter.pdf',
    'https://s2.q4cdn.com/299287126/files/doc_financials/2021/ar/Amazon-2020-Shareholder-Letter-and-1997-Shareholder-Letter.pdf',
    'https://s2.q4cdn.com/299287126/files/doc_financials/2020/ar/2019-Shareholder-Letter.pdf'
]

filenames = [
    'AMZN-2022-Shareholder-Letter.pdf',
    'AMZN-2021-Shareholder-Letter.pdf',
    'AMZN-2020-Shareholder-Letter.pdf',
    'AMZN-2019-Shareholder-Letter.pdf'
]

data_root_2 = "./data_2/" # destination directory

for idx, url in enumerate(urls):
    file_path = data_root_2 + filenames[idx]
    urlretrieve(url, file_path)

#### Upload data to S3 Bucket data source

In [14]:
# Upload data to s3 to the bucket that was configured as a data source to the knowledge base
s3_client = boto3.client("s3")
def uploadDirectory(path,bucket_name):
        for root,dirs,files in os.walk(path):
            for file in files:
                s3_client.upload_file(os.path.join(root,file),bucket_name,file)

uploadDirectory(data_root_1, bucket_name_1)

# You can comment below line if you don't want to use multiple data sources
uploadDirectory(data_root_2, bucket_name_2) 

## Create Knowledge Base
Steps:
- initialize Open search serverless configuration which will include collection ARN, index name, vector field, text field and metadata field.
- initialize the Titan embeddings model ARN, as this will be used to create the embeddings for each of the text chunks.

In [15]:
opensearchServerlessConfiguration = {
            "collectionArn": collection["createCollectionDetail"]['arn'],
            "vectorIndexName": index_name,
            "fieldMapping": {
                "vectorField": "vector",
                "textField": "text",
                "metadataField": "text-metadata"
            }
        }

# The embedding model used by Bedrock to embed ingested documents, and realtime prompts
embeddingModelArn = f"arn:aws:bedrock:{region_name}::foundation-model/amazon.titan-embed-text-v1"

name = f"bedrock-sample-knowledge-base-{suffix}"
description = "Amazon shareholder letter knowledge base."
roleArn = bedrock_kb_execution_role_arn


Provide the above configurations as input to the `create_knowledge_base` method, which will create the Knowledge base.

In [16]:
# Create a KnowledgeBase
from retrying import retry

@retry(wait_random_min=1000, wait_random_max=2000,stop_max_attempt_number=7)
def create_knowledge_base_func():
    create_kb_response = bedrock_agent_client.create_knowledge_base(
        name = name,
        description = description,
        roleArn = roleArn,
        knowledgeBaseConfiguration = {
            "type": "VECTOR",
            "vectorKnowledgeBaseConfiguration": {
                "embeddingModelArn": embeddingModelArn
            }
        },
        storageConfiguration = {
            "type": "OPENSEARCH_SERVERLESS",
            "opensearchServerlessConfiguration":opensearchServerlessConfiguration
        }
    )
    return create_kb_response["knowledgeBase"]

In [17]:
try:
    kb = create_knowledge_base_func()
except Exception as err:
    print(f"{err=}, {type(err)=}")

In [18]:
pp.pprint(kb)

{ 'createdAt': datetime.datetime(2024, 6, 10, 17, 2, 21, 648344, tzinfo=tzutc()),
  'description': 'Amazon shareholder letter knowledge base.',
  'knowledgeBaseArn': 'arn:aws:bedrock:us-east-1:017444429555:knowledge-base/OI4NXPPZYY',
  'knowledgeBaseConfiguration': { 'type': 'VECTOR',
                                  'vectorKnowledgeBaseConfiguration': { 'embeddingModelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-embed-text-v1'}},
  'knowledgeBaseId': 'OI4NXPPZYY',
  'name': 'bedrock-sample-knowledge-base-867',
  'roleArn': 'arn:aws:iam::017444429555:role/AmazonBedrockExecutionRoleForKnowledgeBase_564',
  'status': 'CREATING',
  'storageConfiguration': { 'opensearchServerlessConfiguration': { 'collectionArn': 'arn:aws:aoss:us-east-1:017444429555:collection/dlukh8g4zwzhq8e5mmq3',
                                                                   'fieldMapping': { 'metadataField': 'text-metadata',
                                                                       

In [19]:
# Get KnowledgeBase 
get_kb_response = bedrock_agent_client.get_knowledge_base(knowledgeBaseId = kb['knowledgeBaseId'])

Next we need to create data source(s), which will be associated with the knowledge base created above. Once the data source(s) is ready, we can then start to ingest the documents.

## Create Data Source(s)
Steps:
- initialize chunking strategy, based on which KB will split the documents into pieces of size equal to the chunk size mentioned in the `chunkingStrategyConfiguration`.
- initialize the s3 configuration, which will be used to create the data source object later.


In [20]:
# Function to create KB
def create_ds(bucket_names):
    ds_list=[]
    for bucket_name in bucket_names:
        # Ingest strategy - How to ingest data from the data source
        chunkingStrategyConfiguration = {
            "chunkingStrategy": "FIXED_SIZE",
            "fixedSizeChunkingConfiguration": {
                "maxTokens": 512,
                "overlapPercentage": 20
            }
        }
        
        # The data source to ingest documents from, into the OpenSearch serverless knowledge base index
        s3Configuration = {
            "bucketArn": f"arn:aws:s3:::{bucket_name}",
            # "inclusionPrefixes":["*.*"] # you can use this if you want to create a KB using data within s3 prefixes.
            }
        # Create a DataSource in KnowledgeBase 
        create_ds_response = bedrock_agent_client.create_data_source(
            name = f'{name}-{bucket_name}',
            description = description,
            knowledgeBaseId = kb['knowledgeBaseId'],
            dataSourceConfiguration = {
                "type": "S3",
                "s3Configuration":s3Configuration
            },
            vectorIngestionConfiguration = {
                "chunkingConfiguration": chunkingStrategyConfiguration
            }
        )
        ds = create_ds_response["dataSource"]
        pp.pprint(ds)
        ds_list.append(ds)
    return ds_list

In [21]:
data_sources = create_ds(bucket_names)

{ 'createdAt': datetime.datetime(2024, 6, 10, 17, 2, 22, 378506, tzinfo=tzutc()),
  'dataDeletionPolicy': 'DELETE',
  'dataSourceConfiguration': { 's3Configuration': { 'bucketArn': 'arn:aws:s3:::bedrock-kb-us-east-1-017444429555-1'},
                               'type': 'S3'},
  'dataSourceId': 'WV27YM9JYS',
  'description': 'Amazon shareholder letter knowledge base.',
  'knowledgeBaseId': 'OI4NXPPZYY',
  'name': 'bedrock-sample-knowledge-base-867-bedrock-kb-us-east-1-017444429555-1',
  'status': 'AVAILABLE',
  'updatedAt': datetime.datetime(2024, 6, 10, 17, 2, 22, 378506, tzinfo=tzutc()),
  'vectorIngestionConfiguration': { 'chunkingConfiguration': { 'chunkingStrategy': 'FIXED_SIZE',
                                                               'fixedSizeChunkingConfiguration': { 'maxTokens': 512,
                                                                                                   'overlapPercentage': 20}}}}
{ 'createdAt': datetime.datetime(2024, 6, 10, 17, 2, 22, 577

In [22]:
# Get DataSource 
for idx, ds in enumerate(data_sources):
    print(bedrock_agent_client.get_data_source(knowledgeBaseId = kb['knowledgeBaseId'], dataSourceId = ds["dataSourceId"]))
    print(" ")

{'ResponseMetadata': {'RequestId': '5867b5a8-4bfe-4685-9dde-8651464d1fc5', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Mon, 10 Jun 2024 17:02:22 GMT', 'content-type': 'application/json', 'content-length': '641', 'connection': 'keep-alive', 'x-amzn-requestid': '5867b5a8-4bfe-4685-9dde-8651464d1fc5', 'x-amz-apigw-id': 'ZKSw1HzIoAMEO4w=', 'x-amzn-trace-id': 'Root=1-6667319e-70f8eb1f5b6d7ac733c0aefc'}, 'RetryAttempts': 0}, 'dataSource': {'createdAt': datetime.datetime(2024, 6, 10, 17, 2, 22, 378506, tzinfo=tzutc()), 'dataDeletionPolicy': 'DELETE', 'dataSourceConfiguration': {'s3Configuration': {'bucketArn': 'arn:aws:s3:::bedrock-kb-us-east-1-017444429555-1'}, 'type': 'S3'}, 'dataSourceId': 'WV27YM9JYS', 'description': 'Amazon shareholder letter knowledge base.', 'knowledgeBaseId': 'OI4NXPPZYY', 'name': 'bedrock-sample-knowledge-base-867-bedrock-kb-us-east-1-017444429555-1', 'status': 'AVAILABLE', 'updatedAt': datetime.datetime(2024, 6, 10, 17, 2, 22, 378506, tzinfo=tzutc()), 'vectorIng

### Start ingestion job
Once the KB and data source(s) created, we can start the ingestion job for each data source.
During the ingestion job, KB will fetch the documents in the data source, pre-process it to extract text, chunk it based on the chunking size provided, create embeddings of each chunk and then write it to the vector database, in this case OSS.

In [23]:
ingest_jobs=[]
# Start an ingestion job
for idx, ds in enumerate(data_sources):
    try:
        start_job_response = bedrock_agent_client.start_ingestion_job(knowledgeBaseId = kb['knowledgeBaseId'], dataSourceId = ds["dataSourceId"])
        job = start_job_response["ingestionJob"]
        print(f"job {idx} started successfully\n")
        
        while(job['status']!='COMPLETE' ):
            get_job_response = bedrock_agent_client.get_ingestion_job(
              knowledgeBaseId = kb['knowledgeBaseId'],
                dataSourceId = ds["dataSourceId"],
                ingestionJobId = job["ingestionJobId"]
          )
            job = get_job_response["ingestionJob"]
        pp.pprint(job)
        interactive_sleep(40)

        ingest_jobs.append(job)
    except Exception as e:
        print(f"Couldn't start {idx} job.\n")
        print(e)
    
    

job 0 started successfully

{ 'dataSourceId': 'WV27YM9JYS',
  'failureReasons': [ '["Encountered error: Ignored 4 files as their file '
                      'format was not supported. [Files: .DS_Store, '
                      'lambda_function.zip, jeep.jpg, car-listings.zip]. Call '
                      'to Amazon S3 Source did not succeed."]'],
  'ingestionJobId': 'APTNMSAM9K',
  'knowledgeBaseId': 'OI4NXPPZYY',
  'startedAt': datetime.datetime(2024, 6, 10, 17, 2, 24, 658955, tzinfo=tzutc()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 4,
                  'numberOfDocumentsScanned': 9,
                  'numberOfMetadataDocumentsModified': 0,
                  'numberOfMetadataDocumentsScanned': 0,
                  'numberOfModifiedDocumentsIndexed': 0,
                  'numberOfNewDocumentsIndexed': 5},
  'status': 'COMPLETE',
  'updatedAt': datetime.datetime(2024, 6, 10, 17, 2, 58, 665644, tzinfo=tzutc())}
Done!................

In [24]:
# Print the knowledge base Id in bedrock, that corresponds to the Opensearch index in the collection we created before, we will use it for the invocation later
kb_id = kb["knowledgeBaseId"]
pp.pprint(kb_id)

'OI4NXPPZYY'


In [25]:
# keep the kb_id for invocation later in the invoke request
%store kb_id

Stored 'kb_id' (str)


## Test the knowledge base
### Note: If you plan to run any following notebooks, you can skip this section
### Using RetrieveAndGenerate API
Behind the scenes, RetrieveAndGenerate API converts queries into embeddings, searches the knowledge base, and then augments the foundation model prompt with the search results as context information and returns the FM-generated response to the question. For multi-turn conversations, Knowledge Bases manage short-term memory of the conversation to provide more contextual results.

The output of the RetrieveAndGenerate API includes the generated response, source attribution as well as the retrieved text chunks.

In [26]:
# # try out KB using RetrieveAndGenerate API
bedrock_agent_runtime_client = boto3.client("bedrock-agent-runtime", region_name=region_name)
# Lets see how different Anthropic models responds to the input text we provide
claude_model_ids = [ ["Claude 3 Sonnet", "anthropic.claude-3-sonnet-20240229-v1:0"]]

In [27]:
def ask_bedrock_llm_with_knowledge_base(query: str, model_arn: str, kb_id: str) -> str:
    response = bedrock_agent_runtime_client.retrieve_and_generate(
        input={
            'text': query
        },
        retrieveAndGenerateConfiguration={
            'type': 'KNOWLEDGE_BASE',
            'knowledgeBaseConfiguration': {
                'knowledgeBaseId': kb_id,
                'modelArn': model_arn
            }
        },
    )

    return response

In [28]:
query = "What is Amazon's doing in the field of generative AI?"

for model_id in claude_model_ids:
    model_arn = f'arn:aws:bedrock:{region_name}::foundation-model/{model_id[1]}'
    response = ask_bedrock_llm_with_knowledge_base(query, model_arn, kb_id)
    generated_text = response['output']['text']
    citations = response["citations"]
    contexts = []
    for citation in citations:
        retrievedReferences = citation["retrievedReferences"]
        for reference in retrievedReferences:
            contexts.append(reference["content"]["text"])
    print(f"---------- Generated using {model_id[0]}:")
    pp.pprint(generated_text )
    print(f'---------- The citations for the response generated by {model_id[0]}:')
    pp.pprint(contexts)
    print()

---------- Generated using Claude 3 Sonnet:
('Amazon has been working on large language models (LLMs) for generative AI '
 'for some time. The company believes generative AI will transform and improve '
 'virtually every customer experience, and is investing substantially in these '
 'models across its consumer, seller, brand, and creator experiences. Amazon '
 'is also democratizing this technology through AWS so that companies of all '
 'sizes can leverage generative AI. AWS is offering machine learning chips '
 'like Trainium and Inferentia that provide high performance and low cost for '
 'training and running LLMs in production. AWS also enables companies to '
 "choose from various LLMs and build applications with AWS's security, privacy "
 'and other features. One example application is AWS CodeWhisperer, which '
 'generates code suggestions in real-time to boost developer productivity.')
---------- The citations for the response generated by Claude 3 Sonnet:
[ 'This shift was dr

### Retrieve API
Retrieve API converts user queries into embeddings, searches the knowledge base, and returns the relevant results, giving you more control to build custom workﬂows on top of the semantic search results. The output of the Retrieve API includes the the retrieved text chunks, the location type and URI of the source data, as well as the relevance scores of the retrievals.

In [ ]:
# retrieve api for fetching only the relevant context.
relevant_documents = bedrock_agent_runtime_client.retrieve(
    retrievalQuery= {
        'text': query
    },
    knowledgeBaseId=kb_id,
    retrievalConfiguration= {
        'vectorSearchConfiguration': {
            'numberOfResults': 3 # will fetch top 3 documents which matches closely with the query.
        }
    }
)

In [ ]:
pp.pprint(relevant_documents["retrievalResults"])

### Clean up
Please make sure to comment the below section if you are planning to use the Knowledge Base that you created above for building your RAG application. If you only wanted to try out creating the KB using SDK, then please make sure to delete all the resources that were created as you will be incurred cost for storing documents in OSS index.

In [ ]:

# Delete KnowledgeBase
for idx, ds in enumerate(data_sources):
    bedrock_agent_client.delete_data_source(dataSourceId = ds["dataSourceId"], knowledgeBaseId=kb['knowledgeBaseId'])
bedrock_agent_client.delete_knowledge_base(knowledgeBaseId=kb['knowledgeBaseId'])
oss_client.indices.delete(index=index_name)
aoss_client.delete_collection(id=collection_id)
aoss_client.delete_access_policy(type="data", name=access_policy['accessPolicyDetail']['name'])
aoss_client.delete_security_policy(type="network", name=network_policy['securityPolicyDetail']['name'])
aoss_client.delete_security_policy(type="encryption", name=encryption_policy['securityPolicyDetail']['name'])

In [ ]:
# delete role and policies
from utility import delete_iam_role_and_policies
delete_iam_role_and_policies()